In [1]:
# Useful imports
import numpy as np
from matplotlib.pyplot import figure, show
from astropy.io import fits
from pathlib import Path
from mpl_toolkits.axes_grid1 import make_axes_locatable
import astroalign as aa

In [2]:
# Gets masterbias and masterflats

# path to masters
path = "/net/virgo01/data/users/mpucovsky/OAProject/2025-05-05/DataReduction"
pathObj = Path(path)

masterFlats = {}
for f in pathObj.iterdir():
    hduList = fits.open(f)
    head = hduList[0].header
    data = hduList[0].data
    name = f.name
    if "MBias" in name:
        if "MBias3" in name:
            MBias3 = data
        else:
            MBias = data
    else:
        if "Lum" in name:
            masterFlats["MFlatLum3"] = data
        elif "3" in name:
            masterFlats[f"{name[4:11]}"] = data
        else:
            masterFlats[f"{name[4:10]}"] = data
    hduList.close()


In [3]:
# gets and reduces all images taken

def Reduce(data, head):
    """Reduces input image"""
    if data.shape == (1173, 1552): # 3x3 binning
        data -= MBias3
        if head["FILTER"] == "Lum":
            data /= masterFlats["MFlatLum3"]
        else:
            data /= masterFlats[f"MFlat{head['FILTER'][:1]}3"]
        return data
    elif data.shape == (3520, 4656): # 1x1 binning
        data -= MBias
        if head["FILTER"] == "Lum": # There is no 1x1 binning Lum, 
            return None    # however a file keeps tripping this up (presumably a standard star field, alternatively the "test image")
        else:              # data taken in Lum was using 3x3 binning, so it is alright to exclude this.
            data /= masterFlats[f"MFlat{head['FILTER'][:1]}"]
        return data

# path to data
path = "/net/virgo01/data/users/mpucovsky/OAProject/2025-05-05"
pathObj = Path(path)

reducedIm = {}
for f in pathObj.iterdir():
    if not f.is_dir():
        hduList = fits.open(f)
        head = hduList[0].header
        name = f.name
        if head["IMAGETYP"].upper() == "LIGHT" and head["FILTER"] != "ND1.2":
            data = hduList[0].data.astype(float)
            reducedIm[name] = Reduce(data, head)
        elif head["IMAGETYP"].upper() == "FLAT" and head["EXPTIME"] == 30.0:
            data = hduList[0].data.astype(float)
            reducedIm[name] = Reduce(data, head)
        hduList.close

print(reducedIm.keys())

/tmp/ipykernel_3675858/2929769705.py:10: RuntimeWarning: invalid value encountered in divide
  data /= masterFlats[f"MFlat{head['FILTER'][:1]}3"]
/tmp/ipykernel_3675858/2929769705.py:8: RuntimeWarning: invalid value encountered in divide
  data /= masterFlats["MFlatLum3"]


dict_keys(['LDST_250505-0002_B3.fit', 'LDST_250505-0007_Star_G3.fit', 'LDST_250505-0008_FlatR3.fit', 'LDST_250505-0031_R3.fit', 'LDST_250505-0033_G3.fit', 'LDST_250505-0008_Star_G3.fit', 'LDST_250505-0024_Lum3.fit', 'LDST_250505-0028_G3.fit', 'LDST_250505-0019_B3.fit', 'LDST_250505-0005_Star_G.fit', 'LDST_250505-0014_B3.fit', 'LDST_250505-0034_Lum3.fit', 'LDST_250505-0001_Star_B3.fit', 'LDST_250505-0005_Star_R3.fit', 'LDST_250505-0027_R3.fit', 'LDST_250505-0025_G3.fit', 'LDST_250505-0010_R3.fit', 'LDST_250505-0012_G3.fit', 'LDST_250505-0023_B3.fit', 'LDST_250505-0005_Star_R.fit', 'LDST_250505-0038_B3.fit', 'LDST_250505-0009_G3.fit', 'LDST_250505-0004_Lum3.fit', 'LDST_250505-0006_R3.fit', 'LDST_250505-0004_G3.fit', 'LDST_250505-0014_Lum3.fit', 'LDST_250505-0035_B3.fit', 'LDST_250505-0028_R3.fit', 'LDST_250505-0003_Star_B3.fit', 'LDST_250505-0007_Star_R3.fit', 'LDST_250505-0033_Lum3.fit', 'LDST_250505-0031_G3.fit', 'LDST_250505-0008_FlatG3.fit', 'LDST_250505-0008_Star_R3.fit', 'LDST_2505

In [4]:
# Aligns all starfields and all images of WASP-135

def Align(source, Aligner, f):
    try:
        aligned, footprint = aa.register(source, Aligner)
        return aligned
    except:
        print(f"{f}: Failed")
        return source

# R,G,B are all similar enough that one G image will be used as a reference for all of them
# reference images on which to align starfields
starAlign = reducedIm["LDST_250505-0001_Star_G.fit"]

starAlign3 = reducedIm["LDST_250505-0001_Star_G3.fit"]

# reference image on which to align images of WASP-135
# Lum is different enough to warrant its own reference image
objectAlign = reducedIm["LDST_250505-0001_G3.fit"]

objectAlignLum = reducedIm["LDST_250505-0001_Lum3.fit"]


alignedImages = {}
for f in reducedIm:
    data = reducedIm[f]
    if "Star" in f or "Flat" in f:
        if data.shape == (1173, 1552): # 3x3 binning
            alignedImages[f] = Align(data, starAlign3, f)
        elif data.shape == (3520, 4656): # 1x1 binning
            alignedImages[f] = Align(data, starAlign, f)
    else:
        if "Lum" in f:
            alignedImages[f] = Align(data, objectAlignLum, f)
        else:
            alignedImages[f] = Align(data, objectAlign, f)
            
print(alignedImages.keys())

TestLocation.fit: Failed
dict_keys(['LDST_250505-0002_B3.fit', 'LDST_250505-0007_Star_G3.fit', 'LDST_250505-0008_FlatR3.fit', 'LDST_250505-0031_R3.fit', 'LDST_250505-0033_G3.fit', 'LDST_250505-0008_Star_G3.fit', 'LDST_250505-0024_Lum3.fit', 'LDST_250505-0028_G3.fit', 'LDST_250505-0019_B3.fit', 'LDST_250505-0005_Star_G.fit', 'LDST_250505-0014_B3.fit', 'LDST_250505-0034_Lum3.fit', 'LDST_250505-0001_Star_B3.fit', 'LDST_250505-0005_Star_R3.fit', 'LDST_250505-0027_R3.fit', 'LDST_250505-0025_G3.fit', 'LDST_250505-0010_R3.fit', 'LDST_250505-0012_G3.fit', 'LDST_250505-0023_B3.fit', 'LDST_250505-0005_Star_R.fit', 'LDST_250505-0038_B3.fit', 'LDST_250505-0009_G3.fit', 'LDST_250505-0004_Lum3.fit', 'LDST_250505-0006_R3.fit', 'LDST_250505-0004_G3.fit', 'LDST_250505-0014_Lum3.fit', 'LDST_250505-0035_B3.fit', 'LDST_250505-0028_R3.fit', 'LDST_250505-0003_Star_B3.fit', 'LDST_250505-0007_Star_R3.fit', 'LDST_250505-0033_Lum3.fit', 'LDST_250505-0031_G3.fit', 'LDST_250505-0008_FlatG3.fit', 'LDST_250505-0008

In [7]:
# saves reduced & aligned images
path = "/net/virgo01/data/users/mpucovsky/OAProject/2025-05-05"

for f in alignedImages:
    if f != "TestLocation.fit":
        hduList = fits.open(path+"/"+f)
        head = hduList[0].header
        primary = fits.PrimaryHDU(alignedImages[f], head)
        hdu = fits.HDUList([primary])
        splitF = f.split("-")
        pathW = path+"/Reduced/Reduced_"+splitF[1]
        hdu.writeto(pathW, overwrite=True)
        hduList.close()